In [1]:
import datetime, time, math, os, json, shutil, asyncio, networkx as nx
from utils.gitutils import create_pull_request, clone_repo, create_branch, wipe_repo, prepare_repo
from utils.agent import Agent, GenerationConfig, Interaction, Team
from utils.stringutils import arr_from_sep_string, extract_markdown_blocks, markdown_to_dict, find_most_similar_substring, skwonk
from utils.filetreeutils import FileTree, write_file_tree
from utils.listutils import flatten
from utils.frameworkutils import DartAnalyzer
from dotenv import load_dotenv
from utils.graphutils import loose_level_order, collapsed_level_order
from utils.frameworkutils import Framework

In [2]:
load_dotenv()  # Load environment variables from .env

GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')
API_KEY = os.getenv('API_KEY')

In [3]:
repo_url = 'https://github.com/6165-MSET-CuttleFish/TeamTrack'
source = Framework.FLUTTER
feature = f"""In the Match Editor View, animate the gradient background so the color gradually glows and fades.
"""

In [4]:
# Clone the repo and make a new branch
repo = clone_repo(repo_url)
base_branch = repo.active_branch.name
created_branch = f"modification-{base_branch}"
local_repo_path = str(repo.working_dir)
working_dir_path = f'{local_repo_path}\\{source.get_working_dir()}'

In [5]:
source_swe_summarizer = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="source_swe_analyzer",
    generation_config=GenerationConfig(temperature=0.7),
    system_prompt=f'''You are a {source} software engineer. Your job is to summarize the functionality of provided code files. Structure your response as follows:

# filename.ext:

$7 sentence summary of the functionality/contents of the file$
'''
)
pm = Agent(
    model_name="gemini-1.5-pro-001",
    api_key=API_KEY,
    name="pm",
    generation_config=GenerationConfig(temperature=0.3),
    system_prompt=f"""You are a high-level technical project manager tasked with the project of adding functionality to a {source} repository."""
)
source_swe_coder = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="source_swe_coder",
    generation_config=GenerationConfig(temperature=0.5),
    system_prompt=f"You are a {source} software engineer. Your job is to write code to implement a specific feature. Respond with code and nothing else. No explanations needed."
)
source_swe_editor = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="source_swe_editor",
    generation_config=GenerationConfig(temperature=0.3),
    system_prompt=f"""You are a {source} software engineer tasked with editing a large code file to implement changes in accordance with the instructions you will receive in the following prompts. Large edits at once are NOT the way. We will solve this problem by dividing the edit into smaller sub-edits (no more than 10 lines per edit). Structure your response as follows:

# Step 1
$Brief description of the first sub-change$
## Old Snippet 1
```
$short old code snippet that needs to changed$
```
## New Code Snippet 1
```
$the new code snippet needed to fulfill the prompt$
```
...
...
# Step N
$Brief description of the Nth sub-change$
## Old Code Snippet N
```
$...$
```
## New Code Snippet N
```
$...$
```

MAKE SURE THAT EACH SNIPPET IS SHORT (MY JOB DEPENDS ON IT).
Additionally, DO NOT RESPOND WITH A DIFF FILE OR IN A DIFFERENT FORMAT, MY JOB ALSO DEPENDS ON IT"""
)

team = Team(source_swe_summarizer, pm, source_swe_coder, source_swe_editor)

In [6]:
analyzer = source.get_analyzer(working_dir_path)

source_dependency_graph = analyzer.buildDependencyGraph()

In [7]:
# loose level order
eval_order = loose_level_order(source_dependency_graph)[::-1]
for level in eval_order:
    print(level)
print(f"Number of levels: {len(eval_order)}")

['models\\StatConfig.dart', 'models\\AppModel.dart']
['models\\GameModel.dart']
['models\\ScoreModel.dart']
['components\\misc\\PlatformGraphics.dart', 'functions\\Extensions.dart']
['components\\users\\PFP.dart', 'components\\statistics\\BarGraph.dart']
['providers\\Auth.dart', 'components\\users\\UsersRow.dart', 'components\\scores\\ScoreSummary.dart', 'functions\\Statistics.dart', 'components\\scores\\Incrementor.dart']
['views\\home\\match\\MatchView.dart']
['functions\\Functions.dart', 'components\\statistics\\PercentChange.dart']
['models\\Change.dart', 'components\\scores\\ScoreTimeline.dart', 'components\\scores\\ScoreRangeSummary.dart']
['views\\home\\match\\ExampleMatchRow.dart', 'components\\misc\\CardView.dart', 'components\\scores\\ScoringElementStats.dart', 'components\\statistics\\CheckList.dart', 'components\\misc\\EmptyList.dart', 'views\\home\\change\\ChangeRow.dart', 'views\\home\\change\\ChangeConfig.dart', 'views\\home\\match\\MatchConfig.dart', 'views\\home\\match

In [8]:
(source_file_tree := FileTree.from_dir(working_dir_path))

├── api\
│   ├── README.MD
├── components\
│   ├── misc\
│   │   ├── CardView.dart
│   │   ├── Collapsible.dart
│   │   ├── EmptyList.dart
│   │   ├── InfoPills.dart
│   │   ├── PlatformGraphics.dart
│   ├── scores\
│   │   ├── Incrementor.dart
│   │   ├── ScoreCard.dart
│   │   ├── ScoreRangeSummary.dart
│   │   ├── ScoreSummary.dart
│   │   ├── ScoreTimeline.dart
│   │   ├── ScoringElementStats.dart
│   ├── statistics\
│   │   ├── BarGraph.dart
│   │   ├── CheckList.dart
│   │   ├── PercentChange.dart
│   ├── users\
│   │   ├── PFP.dart
│   │   ├── UsersRow.dart
│   ├── AutonomousDrawingTool.dart
├── functions\
│   ├── APIMethods.dart
│   ├── Extensions.dart
│   ├── Functions.dart
│   ├── GPTModel.dart
│   ├── ResponseModel.dart
│   ├── Statistics.dart
├── models\
│   ├── AppModel.dart
│   ├── Change.dart
│   ├── GameModel.dart
│   ├── GPTModel.dart
│   ├── ScoreModel.dart
│   ├── StatConfig.dart
├── providers\
│   ├── Auth.dart
│   ├── PushNotifications.dart
│   ├── Theme.dart
│   ├

In [9]:
# Summarize the original repo
from tqdm import tqdm

async def summarize_group(group):
    async def summarize(node):
        name = source_file_tree.nodes[node]['name']
        content = source_file_tree.nodes[node]['content']
        message = f"{name}\n```\n{content}\n```"
        return await team.async_chat_with_agent(
            agent_name='source_swe_analyzer',
            message=message,
            context_keys=[f"summary_{neighbor}" for neighbor in source_dependency_graph[node]],
            save_keys=[f"summary_{node}", "all"],
            prompt_title=f"Summary of {node}"
            )
    tasks = [summarize(node) for node in group]
    responses = await asyncio.gather(*tasks)
    for i, response in enumerate(responses):
        source_file_tree.nodes[group[i]]["summary"] = response

for level in tqdm(eval_order):
    await summarize_group(level)
    time.sleep(0.5)

100%|██████████| 19/19 [01:02<00:00,  3.27s/it]


In [10]:
prompt = f"""This is the file tree for the original {source} repo:
```
{source.get_working_dir()}\\
{source_file_tree}
```
Given the prior context, summarize the functionality of the entire repository succinctly.
"""
print(team.chat_with_agent('source_swe_analyzer', prompt, context_keys=["all"], save_keys=["all"], prompt_title=f"""Repository Summary.
File Tree provided:
```
{source.get_working_dir()}\\
{source_file_tree}
```
"""))

The repository implements a Flutter application called TeamTrack, designed for managing and analyzing data related to FIRST Tech Challenge (FTC) robotics events. 

The app provides features for:

* **Event Management:**
    * Creating and managing events (local and remote).
    * Adding and deleting teams.
    * Adding and editing matches.
    * Sharing events with collaborators.
    * Managing permissions for collaborators.
* **Team Management:**
    * Viewing team details, including scores, matches, and statistics.
    * Editing target scores for teams.
    * Drawing autonomous paths for teams.
    * Simulating alliance formations.
    * Recommending alliance partners based on team performance.
* **Match Management:**
    * Viewing match details, including scores, penalties, and active users.
    * Editing scores and penalties.
    * Importing match schedules from images.
* **Data Analysis:**
    * Viewing team statistics and score trends.
    * Analyzing individual scoring elements.

In [11]:
curr_string = '\n'.join([message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict()])
(curr_tokens := pm.model.count_tokens(curr_string).total_tokens)

19552

In [12]:
would_be_string = '\n'.join([source_file_tree.nodes[node]['content'] for node in source_dependency_graph.nodes])
(would_be_tokens := pm.model.count_tokens(would_be_string).total_tokens)

122951

In [13]:
(percent_tokens_reduction := 1 - (curr_tokens / would_be_tokens))

0.8409772998999602

In [14]:
response_format = f"""
You may choose ACTIONS from the following list:
- | WRITE | $FILE_PATH$ | $PROMPT$ (PROMPT should be a brief instruction for how the new FILE_PATH should be WRITTEN)
- | EDIT | $FILE_PATH$ | $PROMPT$ (PROMPT should be a brief instruction for how the FILE_PATH should be EDITED)
- | DELETE | $FILE_PATH$ (Delete the provided file)
----------
Structure your response using the following example:
```
| WRITE | {source.get_working_dir()}\\path\\to\\newfile.ext | Implement ... with the following properties:
    - property1
    - property2
    - dependency1
    - dependency2
| EDIT | {source.get_working_dir()}\\path\\to\\file.ext | Fix the bug ... by doing:
    - step1
    - step2
| DELETE | {source.get_working_dir()}\\path\\to\\some\\file.ext | Reasons for deletion:
    - reason1
    - reason2
```
It is VERY important that you only respond with a high level overview of what to do. The actual implementation will be done by the coder.
"""

# Create new file tree
prompt = f'''
You are to build the following feature:
{feature}
------------------
{response_format}
------------------
You need to follow the response template structure exactly (MY JOB DEPENDS ON IT)
'''
print(response := team.chat_with_agent("pm", prompt, context_keys=["all"], save_keys=["all"], prompt_title="Feature Creation Steps"))

```
| EDIT | lib\views\home\match\MatchView.dart | Implement a gradient animation in the background of the Match Editor View:
    - Use the `AnimatedBuilder` widget to animate the gradient.
    - Create an `AnimationController` to control the animation.
    - Define a `Gradient` object with the desired colors.
    - Use the `Tween` class to animate the gradient's stops.
    - Update the `Gradient` object's stops within the `AnimatedBuilder` based on the animation value.
    - Apply the animated gradient to a `Container` that fills the background of the Match Editor View. 
```



In [15]:
md = extract_markdown_blocks(response)[0]
blocks = md.split("|")
actions = []
i = 0
while i < len(blocks):
    block = blocks[i].strip()
    if block != "WRITE" and block != "EDIT" and block != "DELETE":
        i += 1
        continue
    action, file, prompt = blocks[i:i + 3]
    action = action.strip()
    if file.strip().startswith(source.get_working_dir()):
        file = file.strip()[len(source.get_working_dir()) + 1:]
    else:
        file = file.strip()
    prompt = prompt.strip()
    actions.append((action, file, prompt))
    i += 3
for action in actions:
    print(action)

('EDIT', 'views\\home\\match\\MatchView.dart', "Implement a gradient animation in the background of the Match Editor View:\n    - Use the `AnimatedBuilder` widget to animate the gradient.\n    - Create an `AnimationController` to control the animation.\n    - Define a `Gradient` object with the desired colors.\n    - Use the `Tween` class to animate the gradient's stops.\n    - Update the `Gradient` object's stops within the `AnimatedBuilder` based on the animation value.\n    - Apply the animated gradient to a `Container` that fills the background of the Match Editor View.")


In [16]:
source_swe_coder.model.count_tokens('\n'.join(message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict())).total_tokens

19706

In [17]:
# Cache the context threads
factor = 32768 / source_swe_coder.model.count_tokens('\n'.join(message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict())).total_tokens
# eg. if factor is 3.2, round to 4
factor = math.ceil(factor)
thread = team.context_threads['all'].copy()
thread *= factor
source_swe_coder.model.count_tokens('\n'.join(message['parts'][0] for interaction in thread for message in interaction.to_dict())).total_tokens

39380

In [18]:
def write(file_path, prompt):
    response = source_swe_coder.chat(f"Instructions for {file_path}:\n{prompt}")
    md = extract_markdown_blocks(response)[0]
    path = f"{working_dir_path}\\{file_path}"
    if not os.path.exists(os.path.dirname(path)):
        os.makedirs(os.path.dirname(path))
    with open(path, 'w') as f:
        f.write(md)

def edit(file_path, prompt):
    with open(f"{working_dir_path}\\{file_path}", 'r') as f:
        content = f.read()
    response = source_swe_editor.chat(prompt, custom_context=[Interaction(f"Contents of {file_path}", content)])
    blocks = extract_markdown_blocks(response)
    # even blocks are old code, odd blocks are new code
    old_codes = [block for i, block in enumerate(blocks) if i % 2 == 0]
    new_codes = [block for i, block in enumerate(blocks) if i % 2 == 1]
    for old_code, new_code in zip(old_codes, new_codes):
        with open(f'{working_dir_path}\\{file_path}', 'w') as f:
            f.seek(0)
            f.truncate()
            f.write(skwonk(content, old_code, new_code))

def delete(file_path):
    os.remove(f"{working_dir_path}\\{file_path}")

In [19]:
from utils.errorutils import repeat_until_finish
from tqdm import tqdm

for i in tqdm(range(len(actions))):
    action, file, prompt = actions[i]
    if action == 'WRITE':
        repeat_until_finish(lambda: write(file, prompt), max_retries=2)
    elif action == 'EDIT':
        repeat_until_finish(lambda: edit(file, prompt), max_retries=2)
    elif action == 'DELETE':
        delete(file)

100%|██████████| 1/1 [00:48<00:00, 48.42s/it]
